# AI Filter Training Notebook

This notebook walks through loading trade data, building features, training an XGBoost model, and saving the model artifact.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
import xgboost as xgb

from src.ai_filter import compute_feature_frame, _extract_labels

In [ ]:
data = pd.read_csv('data/trades.csv')
features = compute_feature_frame(data)
labels = _extract_labels(data).reindex(features.index).dropna()
features = features.loc[labels.index]
features.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42, stratify=labels
)
model = xgb.XGBClassifier(
    n_estimators=200, max_depth=4, learning_rate=0.05,
    subsample=0.9, colsample_bytree=0.8,
    objective='binary:logistic', eval_metric='logloss'
)
model.fit(X_train, y_train)

In [ ]:
preds = model.predict(X_test)
proba = model.predict_proba(X_test)[:, 1]
print(classification_report(y_test, preds))
print('ROC AUC:', roc_auc_score(y_test, proba))

In [ ]:
model.save_model('models/ai_filter.json')
print('Saved model to models/ai_filter.json')